In [1]:
# read yaml config

import yaml

config = yaml.load(open("config.yaml"), Loader=yaml.FullLoader)
config

{'decision_analysis_filepaths': 'da_filepaths.txt',
 'corpus_embeddings_path': 'decision_analysis_embeddings.pkl',
 'transcriptions_path': 'transcriptions.txt',
 'query_pdf_path': 'output.pdf'}

In [4]:
files = open(config["decision_analysis_filepaths"]).read().splitlines()
files

['decision_analysis/da-lec1-notes.pdf',
 'decision_analysis/da-lec2-notes.pdf',
 'decision_analysis/da-lec3-notes.pdf',
 'decision_analysis/da-lec4-notes.pdf',
 'decision_analysis/da-lec5-notes.pdf',
 'decision_analysis/da-lec6-notes.pdf',
 'decision_analysis/da-lec7-notes.pdf',
 'decision_analysis/da-lec8-notes.pdf',
 'decision_analysis/da-lec9-notes.pdf',
 'decision_analysis/da-lec10-notes.pdf',
 'decision_analysis/da-lec11-notes.pdf',
 'decision_analysis/da-lec12-notes.pdf']

In [5]:
from lectures_format import DecisionAnalysisLecture

lecture = DecisionAnalysisLecture(files)

In [10]:
transcriptions = lecture.get_transcriptions_from_file(config["transcriptions_path"])
len(transcriptions)

498

In [12]:
import slide_ranking
corpus_embeddings = slide_ranking.load_embeddings(config["corpus_embeddings_path"])

In [14]:
hits = slide_ranking.get_hits("how does promethee method work?", corpus_embeddings, transcriptions)

In [15]:
hits[:5]

[{'corpus_id': 248, 'score': 0.5636867880821228, 'cross-score': 6.841872},
 {'corpus_id': 19, 'score': 0.4326290786266327, 'cross-score': 6.2671432},
 {'corpus_id': 39, 'score': 0.6098626852035522, 'cross-score': 5.791109},
 {'corpus_id': 41, 'score': 0.6389575004577637, 'cross-score': 5.2392817},
 {'corpus_id': 30, 'score': 0.6317201852798462, 'cross-score': 5.1924763}]

In [16]:
lecture.print_pages(hits[:5], config["query_pdf_path"])

In [17]:
import webbrowser

webbrowser.open("output.pdf")

True